In [2411]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [2412]:
df  = pd.read_excel("data/pvasco_todos.xlsx")
data_vasco = df.copy()
#data_vasco.head()

In [2413]:
data_vasco.drop_duplicates(inplace=True)
data_vasco = data_vasco[["Código del contrato","Objeto del contrato","Tipo de contrato","Procedimiento de adjudicación","Criterios de adjudicación","Entidad impulsora","Duración del contrato","Importe de adjudicación","Importe de adjudicación con IVA",'Presupuesto sin IVA', 'Presupuesto con IVA','¿Caracter plurianual?','CPV','División en lotes','Nº de licitadores presentados','¿UTE?']]
data_vasco.head()

,Código del contrato,Objeto del contrato,Tipo de contrato,Procedimiento de adjudicación,Criterios de adjudicación,Entidad impulsora,Duración del contrato,Importe de adjudicación,Importe de adjudicación con IVA,Presupuesto sin IVA,Presupuesto con IVA,¿Caracter plurianual?,CPV,División en lotes,Nº de licitadores presentados,¿UTE?
0,012/15334_00000000000000000000001,Suministro de Gasóleo C para calefacción,Suministros,Abierto,Pluralidad de criterios,Academia Vasca de Policía y Emergencias,17 Meses,525.000,525.000,"433.884,29",525.000,Sí,09134000-7 Gasóleos,No,1.00,No
1,C02/014/2014334_00000000000000000000001,Servicio de limpieza para la Academia Vasca de...,Servicios,Abierto,Pluralidad de criterios,Academia Vasca de Policía y Emergencias,18 Meses,"1.295.401,83",1.567.436,"1.601.652,80",NaN,Sí,90921000-9 Servicios de desinfección y exterminio,No,1.00,No
2,021/15334_00000000000000000000002,Mantenimiento de las instalaciones de calefacc...,Servicios,Abierto,Pluralidad de criterios,Academia Vasca de Policía y Emergencias,21 Meses,43.617,87.500,"72.314,04",87.500,Sí,50720000-8 Servicios de reparación y mantenimi...,No,1.00,Sí
3,028/15334_00000000000000000000001,IMPARTICION DE LA FASE PRESENCIAL DEL CURSO DE...,Servicios,Negociado,Mejor precio,Academia Vasca de Policía y Emergencias,3 Meses,"38.016,52",46.000,"38.016,53",46.000,No,80510000-2 Servicios de formación especializada,No,1.00,No
4,039/15334_00000000000000000000001,OBRAS DE REMODELACION ZONA ATENCION AL ALUMNAD...,Obras,Negociado,Mejor precio,Academia Vasca de Policía y Emergencias,2 Meses,118.459,118.459,"99.173,55",120.000,No,45000000-7 Trabajos de construcción,No,1.00,No


In [2414]:
#CALCULAR LOS NULOS POR COLUMNAS
data_vasco.isna().sum()


Código del contrato                    0
Objeto del contrato                    1
Tipo de contrato                     880
Procedimiento de adjudicación          0
Criterios de adjudicación           8298
Entidad impulsora                      0
Duración del contrato               1408
Importe de adjudicación              321
Importe de adjudicación con IVA       36
Presupuesto sin IVA                10398
Presupuesto con IVA                10338
¿Caracter plurianual?                  0
CPV                                 7545
División en lotes                      0
Nº de licitadores presentados       1684
¿UTE?                                  0
dtype: int64

In [2415]:
#DECLACIÓN DE CONSTANTES
IVA = 1.21
BAJA_MEDIA = 1.7369693554999401 

#DECLARACIÓN DE FUNCIONES


def filtrar_tipo_contrato(data):
    # Eliminar filas con 'Tipo de contrato' NaN
    data.dropna(subset=['Tipo de contrato'], inplace=True)
    
    # Reemplazar 'Privados' con 'Otros' en 'Tipo de contrato'
    data["Tipo de contrato"].replace(["Privados"], "Otros", inplace=True)
    
    # Reemplazar 'Gestión de servicios públicos/concesión de servicios' con 'Gestión de servicios públicos' en 'Tipo de contrato'
    data["Tipo de contrato"].replace("Gestión de servicios públicos/concesión de servicios","Gestión de servicios públicos", inplace=True)
    
    # Eliminar filas donde 'Tipo de contrato' es 'Administrativos especiales'
    data.drop(data[data["Tipo de contrato"] == "Administrativos especiales"].index, inplace=True)
    
    return data

def filtrar_criterios_adjudicacion(data):
    # Reemplazar 'El precio no es el único criterio de adjudicación' con 'Pluralidad de criterios' en 'Criterios de adjudicación'
    data["Criterios de adjudicación"].replace("El precio no es el único criterio de adjudicación", "Pluralidad de criterios", inplace=True)
    
    # Rellenar los valores NaN en 'Criterios de adjudicación' con 'Otros'
    data["Criterios de adjudicación"].fillna("Otros", inplace=True)
    
    return data


#Eliminamos los PROCEDIMIENTOS DE ADJUDICACIÓN que no requieren que la empresa haga una oferta 
def filtrar_procedimientos_adjudicacion(data):
    categorias_a_eliminar = ['Directo/Contrato Menor', 'Directo Emergencia', 'Contratos relativos a la prestación de asistencia sanitaria en supuestos de urgencia']
    data.drop(data[data["Procedimiento de adjudicación"].isin(categorias_a_eliminar)].index, inplace=True)
    data["Procedimiento de adjudicación"].replace("Negociado", "Negociado con Publicidad", inplace=True)

#Normalización del campo de DURACIÓN
def normalizar_duracion(duracion):
    try:
        if isinstance(duracion, str):  # Verificar si es una cadena
            if 'Meses' in duracion:
                return int(duracion.split()[0]) / 12
            elif 'Años' in duracion:
                return int(duracion.split()[0])
            elif 'Semanas' in duracion:
                return int(duracion.split()[0]) / 52.14285714285714
            elif 'Días' in duracion:
                return int(duracion.split()[0]) / 365.25
    except ValueError:
        return pd.NA


#Tratamiento de variables binarias si / no
def transformar_binarios(data):
    data['¿Caracter plurianual?'] = data['¿Caracter plurianual?'].map({'Sí': 1, 'No': 0})
    data['División en lotes'] = data['División en lotes'].map({'Sí': 1, 'No': 0})
    data['¿UTE?'] = data['¿UTE?'].map({'Sí': 1, 'No': 0})
    return data

def convertir_float_importes(data):
    data["Presupuesto con IVA"] = data['Presupuesto con IVA'].str.replace('.', '', regex=True).str.replace(',', '.').astype(float)
    data["Presupuesto sin IVA"] = data['Presupuesto sin IVA'].str.replace('.', '', regex=True).str.replace(',', '.').astype(float)
    data["Importe de adjudicación"] = data['Importe de adjudicación'].str.replace('.', '', regex=True).str.replace(',', '.').astype(float)
    data["Importe de adjudicación con IVA"] = data['Importe de adjudicación con IVA'].str.replace('.', '', regex=True).str.replace(',', '.').astype(float)

    return data

def calcular_importe_adjudicacion_nan(data):
    data["Importe de adjudicación"] = np.where(np.isnan(data["Importe de adjudicación"]), data["Importe de adjudicación con IVA"] / IVA, data["Importe de adjudicación"])
    return data

def unificar_lotes(data):
    data["Código del contrato"] = data["Código del contrato"].str.replace(r'_[^_]*$', '', regex=True)
    data['Importe de adjudicación'] = data.groupby('Código del contrato')['Importe de adjudicación'].transform('sum')
    data = data.drop_duplicates(subset='Código del contrato', keep='first')
    return data


def transformar_duracion_contrato(data):
    data["Duración del contrato"] = data.groupby('CPV')['Duración del contrato'].transform(lambda x: x.fillna(x.median()))
    data['Duración del contrato'] = data['Duración del contrato'].transform(lambda x: x.fillna(x.median()))
    data['Duración del contrato'] = data.groupby('CPV')['Duración del contrato'].transform(lambda x: x.replace(0, x.median()))
    data['Duración del contrato'] = data['Duración del contrato'].transform(lambda x: x.replace(0, x.median()))
    data['Duración del contrato'] = data['Duración del contrato'].astype(float)
    return data

def mapear_entidad_impulsora(data):
    mapeo_valores = {
        'Agencia Vasca del Agua': 'Planificación Territorial, Vivienda y Transportes',
        'Instituto de la Memoria, la Convivencia y los Derechos Humanos': 'Igualdad, Justicia y Políticas Sociales',
        'Kontsumobide-Instituto Vasco de Consumo': 'Turismo, Comercio y Consumo',
        'Instituto Vasco de Administración Pública': 'Gobernanza Pública y Autogobierno',
        'Osalan - Instituto Vasco de Seguridad y Salud Laborales': 'Trabajo y Empleo',
        'Emakunde-Instituto Vasco de la Mujer': 'Igualdad, Justicia y Políticas Sociales',
        'EUSTAT - Instituto Vasco de Estadística': 'Economía y Hacienda',
        'Academia Vasca de Policía y Emergencias': 'Seguridad',
        'Instituto de Alfabetización y Reeuskaldunización de Adultos (HABE)': 'Cultura y Política Lingüística',
        '(Baja) Salud': 'Salud',
        '(Baja) Agencia Vasca del Agua': 'Planificación Territorial, Vivienda y Transportes'
    }
    data["Entidad impulsora"] = data["Entidad impulsora"].replace(mapeo_valores)
    return data

def fill_num_licitadores(data):
    group_col = 'Procedimiento de adjudicación'
    data["Nº de licitadores presentados"] = data.groupby("Procedimiento de adjudicación")["Nº de licitadores presentados"].transform(lambda x: x.fillna(x.median()))
    data["Nº de licitadores presentados"] = data["Nº de licitadores presentados"].astype(int)
    return data

def procesar_outliers(data):
    # Corregir valores atípicos en 'Importe de adjudicación'
    data.at[21536, "Importe de adjudicación"] = 100000.0
    data.at[766, "Presupuesto sin IVA"] = 72479.3
    data.at[9648, "Presupuesto sin IVA"] = 100000.0
    data.at[1858,"Presupuesto sin IVA"] = 118496.0
    data.at[3676,"Presupuesto sin IVA"] = 321696.0
    data.at[4770,"Presupuesto sin IVA"] = 566163.0
     
     
    # Corregir valores atípicos en 'Presupuesto con IVA'
    data.at[3434, "Presupuesto con IVA"] = 30250.0
    
    # Corregir valores atípicos en 'Duración del contrato'
    data.at[20440, 'Duración del contrato'] = 1
    data.at[20454, 'Duración del contrato'] = 0.46
    
    data.drop(index=20450, inplace = True)
    # Eliminar filas donde 'Importe de adjudicación' y 'Importe de adjudicación con IVA' son 0
    data.drop(data[(data["Importe de adjudicación"] == 0) & (data["Importe de adjudicación con IVA"] == 0)].index, inplace=True)
    
    return data


def crear_bins_duracion(data):
    # Definir los límites de los bins en años
    bins = [0, 1, 3, 6, float('inf')]  # en años

    # Definir las etiquetas para cada bin
    labels = ['Menos de 1 año', '1-3 años', '3-6 años', 'Más de 6 años']

    # Aplicar la transformación
    data['Duración del contrato'] = pd.cut(data['Duración del contrato'], bins=bins, labels=labels, right=False)

    return data


def crear_bins_licitadores(data):
    # Definir los límites de los bins en años
    bins = [0, 5, 10, 15, 20, float('inf')]  # en años

    # Definir las etiquetas para cada bin
    labels = ['Menos de 5 licitadores', '5-10 licitadores', '10-15 licitadores', '15-20 licitadores', "Más de 20 licitadores"]

    # Aplicar la transformación
    data['Nº de licitadores presentados'] = pd.cut(data['Nº de licitadores presentados'], bins=bins, labels=labels, right=False)

    return data

def calcular_importe_adjudicación_outliers(data):
    data["Importe de adjudicación"] = np.where(data["Importe de adjudicación"]<2,data["Importe de adjudicación con IVA"] / 1.21, data["Importe de adjudicación"] )
    return data

def eliminar_valores_sospechos_importe(data):
    
    # Crear una máscara para las filas donde 'Importe de adjudicación' es mayor que 'Presupuesto sin IVA'
    mask = data["Importe de adjudicación"] > data["Importe de adjudicación con IVA"]

# Intercambiar los valores en las columnas 'Presupuesto sin IVA' e 'Importe de adjudicación' para estas filas
    data.loc[mask, "Importe de adjudicación"] = data.loc[mask, "Importe de adjudicación con IVA"]

    mask = data["Presupuesto sin IVA"] < data["Importe de adjudicación"]

    # Intercambiar los valores en las columnas para las filas que cumplen con la condición
    data.loc[mask, ["Presupuesto sin IVA", "Importe de adjudicación"]] = data.loc[mask, ["Importe de adjudicación", "Presupuesto sin IVA"]].values   

    condicion = data["Presupuesto sin IVA"] == data["Importe de adjudicación"]

# Establecer el valor de "Presupuesto sin IVA" igual a "Presupuesto con IVA" cuando se cumple la condición
    data.loc[condicion, "Presupuesto sin IVA"] = data.loc[condicion, "Presupuesto con IVA"].fillna(data.loc[condicion, "Presupuesto sin IVA"])

    # Crear una máscara para los valores sospechosos
    mask = (data["Presupuesto sin IVA"] - data["Importe de adjudicación"]) / data["Presupuesto sin IVA"] > 0.7

    # Reemplazar los valores sospechosos en 'Importe de adjudicación' con el 80% del valor en 'Presupuesto sin IVA'
    data.loc[mask, "Importe de adjudicación"] = data.loc[mask, "Presupuesto sin IVA"] * 0.7

    data.drop(data[data["Importe de adjudicación"] <= 15000].index, inplace=True)
    data.drop(data[data["Importe de adjudicación"] >= 1000000].index, inplace=True)
    
    return data




In [2416]:
#EJECUCIÓN
filtrar_tipo_contrato(data_vasco)
filtrar_criterios_adjudicacion(data_vasco)
filtrar_procedimientos_adjudicacion(data_vasco)
data_vasco['CPV'] = data_vasco['CPV'].str.extract(r'(\d{2})')
data_vasco['Duración del contrato'] = data_vasco['Duración del contrato'].apply(normalizar_duracion)
data_vasco = transformar_binarios(data_vasco)
data_vasco = convertir_float_importes(data_vasco)
data_vasco = calcular_importe_adjudicacion_nan(data_vasco)
data_vasco = unificar_lotes(data_vasco)
data_vasco = transformar_duracion_contrato(data_vasco)
data_vasco = mapear_entidad_impulsora(data_vasco)
data_vasco = fill_num_licitadores(data_vasco)
procesar_outliers(data_vasco)
data_vasco = crear_bins_duracion(data_vasco)
data_vasco = crear_bins_licitadores(data_vasco)
calcular_importe_adjudicación_outliers(data_vasco)
eliminar_valores_sospechos_importe(data_vasco)
data_vasco['CPV'] = data_vasco['CPV'].astype(str)
data_vasco.drop(columns=["Objeto del contrato","Importe de adjudicación con IVA","Código del contrato","Presupuesto con IVA"], inplace= True)




In [2418]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
data_vasco.describe()

,Importe de adjudicación,Presupuesto sin IVA,¿Caracter plurianual?,División en lotes,¿UTE?
count,3858.00,3858.00,3858.00,3858.00,3858.00
mean,148150.58,186049.05,0.19,0.12,0.04
std,173948.53,223983.89,0.39,0.32,0.20
min,15021.00,5575.50,0.00,0.00,0.00
25%,39402.50,49658.64,0.00,0.00,0.00
50%,77377.68,95767.10,0.00,0.00,0.00
75%,181530.33,221650.00,0.00,0.00,0.00
max,997500.00,1993740.04,1.00,1.00,1.00


In [2426]:
#data_vasco = data_vasco.reset_index(drop=True)#Se crea un nuevo archivo tipo CSV actualizado 
data_vasco.to_csv("data/pvasco_filtro.csv",index=False)